
#  US Accidents: Batch-Wise Data Processing & Severity Prediction  

## Summary  
The **US Accidents dataset** (~7.7M rows, 2016–2023) is one of the largest open traffic accident datasets.  
A key challenge is that the dataset is **too large to process in memory at once**.  

To solve this, I designed a **batch-wise machine learning pipeline** using `chunksize` and `partial_fit`.  
This approach mimics real-world big data workflows and shows ability to handle **scalability challenges**.  
 
  
  


In [ ]:
# important libaries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
# Ignore all warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
import folium
import networkx as nx
import category_encoders as ce
from sklearn.preprocessing import label_binarize


from sklearn.model_selection import cross_val_score, KFold
# Models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import LinearSVC
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix, roc_auc_score, roc_curve,balanced_accuracy_score,precision_recall_curve,average_precision_score, precision_score,recall_score, auc


In [ ]:
import os

data_path = "/Users/satishmudrakola/.cache/kagglehub/datasets/sobhanmoosavi/us-accidents/versions/13"
print(os.listdir(data_path))

## 2. Data Loading (Batch-Wise)

In [ ]:
# Load dataset in chunks
file_path = "/Users/satishmudrakola/.cache/kagglehub/datasets/sobhanmoosavi/us-accidents/versions/13/US_Accidents_March23.csv"

# Read the dataset
chunksize = 100000
data_iter = pd.read_csv(file_path, chunksize=chunksize)
df = next(data_iter)
df


In [ ]:
df.tail()

In [ ]:
df.info()

In [ ]:
df.describe(include='all')

In [ ]:
# cheacking the missing values
missing_value_counts = df.isnull().sum()
missing_value_counts[:46]

In [ ]:
# percentance of missing values
total_cols = np.prod(df.shape)
total_missing = missing_value_counts.sum()
missing_values_per = (total_missing/total_cols )* 100
print(f'Total percent of data that is missing {missing_values_per:2f}')

### The dataset contains about 9% missing values in total.

However, the missingness is not evenly distributed: while most columns have little or no missing data, a few features are severely affected (very high proportion of NaNs).

In [ ]:
# let's check the missing %  values for each cols 
missing_per_col = (missing_value_counts / len(df)) * 100
missing_per_col = missing_per_col[missing_per_col > 0].sort_values(ascending=False)
missing_per_col.head(7)

In [ ]:
# plotting the missing values
missing_per_col.plot(kind='bar', figsize=(12,6))
plt.title("Percentage of Missing Values per Column")
plt.ylabel("Missing %")
plt.show()

In [ ]:
# memeroy usage of the data
df.memory_usage(deep=True).sum() / 1024**2

In [ ]:
# checking the duplicates
df.duplicated().sum()

In [ ]:
# Separate categorical and numerical columns
cat_cols = df.select_dtypes(include='object').columns
num_cols = df.select_dtypes(exclude='object').columns

print("Categorical columns and their unique values:")
print("*" * 30)
for col in cat_cols:
    print(f"{col}: {df[col].nunique()} unique values")

print("*" * 30)
print("\nNumerical columns:")
print(list(num_cols))

In [ ]:
# class balance
df['Severity'].value_counts()

In [ ]:
#checking all the unique values for categorical columns
for col in cat_cols:
    print(f"\nColumn: {col}")
    print(f"Unique values: {df[col].nunique()}")
    print("Top most frequent values:")
    print(df[col].value_counts().head(5))
    print("*" * 60)

In [ ]:
#checking all the unique values for numeric columns
for col in num_cols:
    print(f"\nColumn: {col}")
    print(f"Unique values: {df[col].nunique()}")
    print("Top most frequent values:")
    print(df[col].value_counts().head(5))
    print("*" * 60)

# Data Loading and Inspection Analysis


The dataset was loaded and inspected for structural issues before analysis. Key steps included:

- **Initial Inspection:** Verified dataset shape, column names, and data types.  
- **Duplicates:** Checked and removed duplicate records to avoid bias.  
- **Data Types:** Converted relevant columns into appropriate datetime objects.  


# Data Handling

In [ ]:
# Drop columns 
cols_to_drop = ['ID', 'Description', 'Street', 'Zipcode', 'Weather_Timestamp',
                'End_Lat', 'End_Lng', 'Wind_Chill(F)', 'Country']
df.drop(columns=cols_to_drop, inplace=True)
df.head()

In [ ]:
# Impute missing numeric values
numeric_impute_median = ['Precipitation(in)', 'Wind_Speed(mph)', 'Visibility(mi)']
for col in numeric_impute_median:
    df[col].fillna(df[col].median(), inplace=True)

In [ ]:
# Impute missing categorical values
categorical_impute_mode = ['Wind_Direction']
for col in categorical_impute_mode:
    df[col].fillna(df[col].mode()[0], inplace=True)

In [ ]:
# Wind_Direction: unify "CALM" / "Calm"
df['Wind_Direction'] = df['Wind_Direction'].str.upper()

In [ ]:
# Weather_Condition: group into main buckets
def normalize_weather(x):
    if pd.isnull(x):
        return 'Unknown'
    x = x.lower()
    if any(word in x for word in ['clear', 'fair']):
        return 'Clear'
    elif any(word in x for word in ['cloud', 'overcast', 'partly']):
        return 'Cloudy'
    elif any(word in x for word in ['rain', 'drizzle', 'shower', 'thunder']):
        return 'Rain'
    elif any(word in x for word in ['snow', 'sleet', 'flurries']):
        return 'Snow'
    elif any(word in x for word in ['fog', 'mist', 'haze']):
        return 'Fog'
    else:
        return 'Other'

In [ ]:
df['Weather_Condition'] = df['Weather_Condition'].apply(normalize_weather)
df['Weather_Condition']

In [ ]:
# Checking missing values
print("\nMissing values per column:")
print(df.isnull().sum())

In [ ]:
#Handling Remaining Missing Values
#Fill small missing values in categorical columns
df['City'].fillna('Unknown', inplace=True)
df['Timezone'].fillna('Unknown', inplace=True)
df['Airport_Code'].fillna('Unknown', inplace=True)
df['Sunrise_Sunset'].fillna('Unknown', inplace=True)
df['Civil_Twilight'].fillna('Unknown', inplace=True)
df['Nautical_Twilight'].fillna('Unknown', inplace=True)
df['Astronomical_Twilight'].fillna('Unknown', inplace=True)

#Impute missing numeric weather features with median
weather_numeric = ['Temperature(F)', 'Humidity(%)', 'Pressure(in)']
for col in weather_numeric:
    df[col].fillna(df[col].median(), inplace=True)

#Once again Verifying all missing values are handleded or not
print("Missing values after imputation:")
print(df.isnull().sum().sum())  # total count

In [ ]:
#Outliers detection
num_cols = df.select_dtypes(include=['int64','float64']).columns.tolist()
num_cols.remove('Severity')  # exclude target

plt.figure(figsize=(15,8))
for i, col in enumerate(num_cols):
    plt.subplot(len(num_cols)//3 + 1, 3, i+1)
    sns.boxplot(y=df[col])
    plt.title(col)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(15,8))
for i, col in enumerate(num_cols):
    plt.subplot(len(num_cols)//3 + 1, 3, i+1)
    sns.histplot(df[col], kde=True, bins=50)
    plt.title(col)
plt.tight_layout()
plt.show()


In [ ]:
#Capping method
for col in num_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    df[col] = np.where(df[col] < lower, lower, df[col])
    df[col] = np.where(df[col] > upper, upper, df[col])


In [ ]:
#Visualize After Outlier Handling
plt.figure(figsize=(15,8))
for i, col in enumerate(num_cols):
    plt.subplot(len(num_cols)//3 + 1, 3, i+1)
    sns.boxplot(y=df[col])
    plt.title(f"{col} (after handling)")
plt.tight_layout()
plt.show()


## Data Handling & Preprocessing

A high-quality dataset is the foundation of any robust machine learning pipeline.  
To ensure integrity and reliability, multiple stages of preprocessing were carried out:

### Missing Value Treatment
- **Numeric features** → Imputed using statistical measures (**mean/median**, depending on distribution).  
- **Categorical features** → Imputed using the most frequent category (**mode**).  
- **Remaining missing values** → Re-examined and handled systematically to prevent data leakage or bias.  

### Outlier Detection & Treatment
- Applied the **Interquartile Range (IQR)** method to identify and cap extreme values in key numerical columns.  
- This ensured that unusual records (e.g., abnormal traffic counts or weather readings) did not distort model learning.  

---

**Insight:**  
After these steps, the dataset became consistent, balanced in scale, and ready for downstream modeling.  

**Takeaway:**  
A structured preprocessing pipeline — combining imputation and outlier treatment — transformed raw accident logs into reliable, high-quality training data, reducing noise and improving model robustness.


# Exploratory Data Analysis

In [ ]:
#Target Variable Analysis
plt.figure(figsize=(6,4))
sns.countplot(x='Severity', data=df, palette="coolwarm")
plt.title('Accident Severity Distribution')
plt.show()

# Percentage distribution
severity_percent = df['Severity'].value_counts(normalize=True) * 100
print("Severity percentage distribution:\n", severity_percent)

In [ ]:
#Numerical Feature Distributions
num_cols = df.select_dtypes(include=['int64','float64']).columns.tolist()
num_cols.remove('Severity')

plt.figure(figsize=(15,8))
for i, col in enumerate(num_cols):
    plt.subplot(len(num_cols)//3 + 1, 3, i+1)
    sns.histplot(df[col], kde=True, bins=50, color='skyblue')
    plt.title(col)
plt.tight_layout()
plt.show()

In [ ]:
# Function for violin plots
def plot_violin(df, feature, target="Severity"):
    plt.figure(figsize=(12,6))
    sns.violinplot(
        x=df[target].astype(str),
        y=df[feature],
        palette="coolwarm",
        inner="quartile"
    )
    plt.title(f"{feature} vs {target}", fontsize=14, pad=10)
    plt.xlabel("Accident Severity")
    plt.ylabel(feature)
    plt.show()

features = ["Distance(mi)","Wind_Speed(mph)"]

for f in features:
    plot_violin(df, f)

In [ ]:
# Convert Start_Time to datetime first
df['Start_Time'] = pd.to_datetime(df['Start_Time'], errors='coerce')

# Extract time-based features BEFORE encoding
df['Hour'] = df['Start_Time'].dt.hour
df['DayOfWeek'] = df['Start_Time'].dt.dayofweek
df['Month'] = df['Start_Time'].dt.month
df['Weekend'] = df['DayOfWeek'].apply(lambda x: 1 if x >= 5 else 0)

#Accidents by Hour
plt.figure(figsize=(10,5))
sns.countplot(x='Hour', data=df, palette='coolwarm', order=sorted(df['Hour'].unique()))
plt.title('Accidents by Hour of Day')
plt.xlabel("Hour of Day (0=Midnight)")
plt.ylabel("Accident Count")
plt.show()


#Accidents by Day of Week
plt.figure(figsize=(8,5))
sns.countplot(x='DayOfWeek', data=df, palette='viridis')
plt.title("Accidents by Day of Week (0=Mon, 6=Sun)")
plt.xlabel("Day of Week")
plt.ylabel("Accident Count")
plt.show()


#Accidents by Month
plt.figure(figsize=(10,5))
sns.countplot(x='Month', data=df, palette='magma')
plt.title("Accidents by Month")
plt.xlabel("Month")
plt.ylabel("Accident Count")
plt.show()


#Weekday vs Weekend
plt.figure(figsize=(6,4))
sns.countplot(x='Weekend', data=df, palette='Set2')
plt.title("Accidents: Weekday vs Weekend")
plt.xlabel("")
plt.ylabel("Accident Count")
plt.show()



In [ ]:
#Heatmap: Severity by Hour & DayOfWeek
hour_day = df.groupby(['Hour','DayOfWeek'])['Severity'].mean().unstack()

plt.figure(figsize=(12,6))
sns.heatmap(hour_day, cmap='coolwarm', annot=True, fmt=".2f", cbar_kws={'label': 'Avg Severity'})
plt.title("Average Severity by Hour & Day of Week")
plt.xlabel("Day of Week (0=Mon, 6=Sun)")
plt.ylabel("Hour of Day")
plt.show()

In [ ]:
# Geospatial Clusters
# Sample for performance
sample_df = df[['Start_Lat','Start_Lng']].dropna().sample(10000)
kmeans = KMeans(n_clusters=6, random_state=42).fit(sample_df)
sample_df['Cluster'] = kmeans.labels_

m = folium.Map(location=[39.8283, -98.5795], zoom_start=4)
for _, row in sample_df.iterrows():
    folium.CircleMarker(
        location=[row['Start_Lat'], row['Start_Lng']],
        radius=3,
        color=f"C{row['Cluster']}",
        fill=True,
        fill_opacity=0.7
    ).add_to(m)
m  # Interactive map in notebook


In [ ]:
#correlation network
numeric_df = df.select_dtypes(include=['float64', 'int64'])
corr = numeric_df.corr()

#Build edges based on correlation threshold
edges = [
    (i, j, corr.loc[i, j]) 
    for i in corr.columns 
    for j in corr.columns 
    if i != j and abs(corr.loc[i, j]) > 0.3
]

#Create graph
G = nx.Graph()
for i, j, w in edges:
    G.add_edge(i, j, weight=w)

#Plot correlation network
plt.figure(figsize=(12, 12))
pos = nx.spring_layout(G, k=0.5, seed=42)
nx.draw(G, pos, with_labels=True, 
        node_size=2000, 
        node_color='skyblue', 
        edge_color='gray', 
        font_size=10)
plt.title("Correlation Network of Numeric Features", fontsize=16)
plt.show()




# Exploratory Data Analysis (EDA)

To extract meaningful insights, multiple stages of exploratory analysis were carried out. The findings are structured below:

---

## 1. Target Variable Analysis
- The dataset shows **imbalanced severity classes**.  
- **Severity 2** accidents dominate, making up the majority of records, followed by **Severity 3**.  
- **Severity 1** and **Severity 4** are comparatively rare.  
- This imbalance must be addressed during modeling (e.g., resampling or class-weighting).  

---

## 2. Feature vs Target Analysis
- **Violin Plots** revealed how numerical features vary with severity:
  - **Distance (mi):** Higher severity accidents are often linked with longer distances.  
  - **Wind Speed (mph):** Extreme wind speeds (both calm and high) can increase severity.  
  - **Hour of Day:** Severity patterns shift depending on time (late-night vs rush-hour).  

---

## 3. Temporal Trends
- **Accidents by Hour:** Clear peaks during **rush hours (7–9 AM, 4–7 PM)**.  
- **Accidents by Month:** Seasonal variation observed, with higher accident frequency in **winter months**, likely due to weather.  
- **Weekday vs Weekend:** Weekdays show significantly more accidents compared to weekends.  
- **Severity Heatmap (Hour × Day of Week):**  
  - Weekday mornings and evenings show the **highest severity concentration**.  
  - Late-night accidents on weekends also lean toward higher severity.  

---

## 4. Geospatial Analysis
- Using **Folium maps**, geospatial clustering highlighted accident hotspots.  
- **Eastern USA (East Coast)** shows **much higher accident density** compared to the West.  
- Clusters are especially concentrated around **urban and densely populated areas** (e.g., Northeast Corridor, California highways).  

---

## 5. Correlation & Feature Relationships
- **Correlation Heatmap:** Checked dependencies among numerical features.  
- **Correlation Network:** Visualized groups of strongly connected features, flagging redundancy.  
- While most features had low correlation, a few weather-related and traffic-related features showed moderate association.  

---

###  Key Insights
1. **Severity Imbalance:** Severity 2 dominates, requiring balancing techniques.  
2. **Temporal Effects:** Rush hours and weekdays drive accident frequency, with severity spikes at night.  
3. **Geospatial Distribution:** Eastern US (especially urban regions) has higher accident density than the West.  
4. **Feature Patterns:** Distance and environmental conditions (wind, time of day) influence accident severity.  
5. **Feature Correlations:** Minimal multicollinearity overall, with some weather-related overlaps.  

---

###  Takeaway
The EDA confirms that accident severity is shaped by **when** (time), **where** (geospatial patterns), and **environmental conditions**.  
By combining violin plots, time-series analysis, folium maps, and correlation networks, we established a multi-perspective understanding of accident dynamics in the US.


# Feature Engineering

In [ ]:
def encode_categorical(df, high_card_threshold=50, top_n=10, use_binary_encoding=True):
    """
    Encode categorical features:
    - High-cardinality -> Label or Binary encoding
    - Low-cardinality -> Top-N One-Hot
    - Binary categorical -> 0/1
    """

    df = df.copy()

    # Detect categorical columns
    cat_cols = df.select_dtypes(include='object').columns.tolist()
    print(f"Categorical columns: {cat_cols}")

    # Separate binary, low, high cardinality
    binary_cols = [col for col in cat_cols if df[col].nunique() == 2]
    high_card_cols = [col for col in cat_cols if df[col].nunique() > high_card_threshold]
    low_card_cols = [col for col in cat_cols if 2 < df[col].nunique() <= high_card_threshold]

    print(f"Binary cols: {binary_cols}")
    print(f"Low-cardinality cols: {low_card_cols}")
    print(f"High-cardinality cols: {high_card_cols}")

    # --------------------------
    # Binary cols -> map to 0/1
    for col in binary_cols:
        df[col] = df[col].astype('category').cat.codes

    # --------------------------
    # Low-cardinality -> Top-N One-Hot
    for col in low_card_cols:
        top_values = df[col].value_counts().nlargest(top_n).index
        df[col] = df[col].apply(lambda x: x if x in top_values else 'Other')
    df = pd.get_dummies(df, columns=low_card_cols, drop_first=True)

    # --------------------------
    # High-cardinality
    if use_binary_encoding and high_card_cols:
        encoder = ce.BinaryEncoder(cols=high_card_cols)
        df = encoder.fit_transform(df)
    else:
        for col in high_card_cols:
            df[col] = LabelEncoder().fit_transform(df[col].astype(str))

    print(f"Shape after encoding: {df.shape}")
    return df

# ----------------------
# Example usage
df_encoded = encode_categorical(df, high_card_threshold=50, top_n=10, use_binary_encoding=True)

In [ ]:
#Time-based Enrichment
df_encoded['Rush_Hour'] = df['Hour'].apply(lambda x: 1 if (7 <= x <= 9) or (16 <= x <= 19) else 0)
df_encoded['Season'] = df['Month'].map({12:'Winter',1:'Winter',2:'Winter',
                                3:'Spring',4:'Spring',5:'Spring',
                                6:'Summer',7:'Summer',8:'Summer',
                                9:'Fall',10:'Fall',11:'Fall'})

df_encoded['Season']

In [ ]:
df_encoded['Rush_Hour']

In [ ]:
df_encoded.columns

In [ ]:
#Road/Infrastructure Features
infra_cols = ['Amenity','Bump','Crossing','Give_Way','Junction','No_Exit',
              'Railway','Roundabout','Station','Stop','Traffic_Calming','Traffic_Signal','Turning_Loop']
df_encoded['Infra_Risk_Score'] = df[infra_cols].sum(axis=1)

df_encoded['Infra_Risk_Score']

In [ ]:
# Adverse Weather: Rain, Snow, Fog, etc.
df_encoded['Adverse_Weather'] = df_encoded[['Weather_Condition_Rain', 
                                            'Weather_Condition_Snow',
                                            'Weather_Condition_Fog']].sum(axis=1)
df_encoded['Visibility_Precip'] = df_encoded['Visibility(mi)'] * (df_encoded['Precipitation(in)'] + 1)
df_encoded['Adverse_Weather']

In [ ]:
df_encoded['Visibility_Precip']

In [ ]:
#Geospatial Features
coords = df_encoded[['Start_Lat','Start_Lng']]
kmeans = KMeans(n_clusters=20, random_state=42)
df_encoded['Accident_Cluster'] = kmeans.fit_predict(coords)

In [ ]:
# Frequency-Based Features
df_encoded['City_Freq'] = df_encoded[['City_0','City_1','City_2','City_3','City_4',
                                      'City_5','City_6','City_7','City_8','City_9']].sum(axis=1)
df_encoded['City_Freq']

In [ ]:
season_dummies = pd.get_dummies(df_encoded['Season'], prefix='Season')
df_encoded = pd.concat([df_encoded.drop(columns=['Season']), season_dummies], axis=1)
df_encoded


In [ ]:
# Boolean columns → convert to int
bool_cols = df_encoded.select_dtypes(include='bool').columns.tolist()
df_encoded[bool_cols] = df_encoded[bool_cols].astype(int)

In [ ]:
# Numeric columns
num_cols = df_encoded.select_dtypes(include=['int64', 'float64']).columns.tolist()
# Remove target if included
target = 'Severity'
if target in num_cols:
    num_cols.remove(target)


In [ ]:
# Target
target = 'Severity'

# Current feature set
X = df.drop(columns=[target, 'Start_Time'])  # drop original datetime & target
y = df[target]

# Numeric columns
num_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Categorical columns
cat_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()

print("Numeric columns:", num_cols[:5], "...")
print("Categorical columns:", cat_cols[:5], "...")


In [ ]:
#Bulding a Preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)
    ]
)



# Feature Engineering

To enrich the dataset and enhance predictive power, several feature engineering techniques were applied:

---

## 1. Temporal Features
- Converted **Start_Time** into a datetime object.
- Derived granular time-based features:
  - **Hour** (0–23)  
  - **DayOfWeek** (0–6)  
  - **Month** (1–12)  
- Constructed domain-driven features:
  - **Rush_Hour:** Binary flag for morning/evening rush periods (7–9 AM, 4–7 PM).  
  - **Season:** Mapped months into categorical seasons (Winter, Spring, Summer, Fall).

---

## 2. Categorical Encoding
- Developed a flexible encoding function:
  - **High-cardinality features:** Label or binary encoding.  
  - **Low-cardinality features:** One-hot encoding (Top-N categories preserved).  
- Example: **City** frequencies were retained using a **frequency-based representation**.  

---

## 3. Road & Infrastructure Features
- Engineered an **Infrastructure Risk Score** by summing binary indicators such as:  
  *Amenity, Crossing, Junction, Traffic Signal, Stop, Railway,* etc.  
- This consolidated multiple sparse features into a single interpretable metric.  

---

## 4. Geospatial Features
- Applied **KMeans clustering** on accident coordinates (`Start_Lat`, `Start_Lng`).  
- Created a new feature **Accident_Cluster** to capture regional accident patterns.  

---

## 5. Frequency-Based Features
- Encoded high-frequency categorical attributes by aggregating their top-N dummy variables.  
- Example: **City_Freq** was computed from top 10 most common cities.  

---

## 6. Final Dataset Construction
- Dropped non-predictive fields such as raw `Start_Time` and the target column.  
- Prepared feature matrix **X** and target **y = Severity**.  
- Final dataset included a **blend of raw, derived, and encoded features**, ensuring both interpretability and predictive richness.

---

### Key Takeaways
- **Temporal context** (hour, day, season) provides strong predictive signals.  
- **Domain-driven features** like Rush Hour, Infrastructure Risk, and Clusters improve interpretability.  
- **Encoding strategies** prevented high-cardinality features (e.g., City) from overwhelming the model.  
- Feature engineering transformed the dataset from a flat accident log into a structured, multi-dimensional representation — critical for model performance.


# Train-Test Split

In [ ]:
#Split features and target
X = df_encoded.drop(columns=['Severity', 'Start_Time'], errors='ignore')
y = df_encoded['Severity']

#Train/Test Split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

#Scale numeric features (optional)
from sklearn.preprocessing import StandardScaler
num_cols = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()
scaler = StandardScaler()
X_train[num_cols] = scaler.fit_transform(X_train[num_cols])
X_test[num_cols] = scaler.transform(X_test[num_cols])

print("Train shape:", X_train.shape)
print("Test shape:", X_test.shape)
print("Any missing values in train:", X_train.isnull().sum().sum())
print("Any missing values in test:", X_test.isnull().sum().sum())


In [ ]:
# Identify current columns in X
print(X_train.columns)


# Modeling

In [ ]:
# Model dictionary
models = {
    "Logistic Regression": LogisticRegression(
        max_iter=2000, solver="lbfgs", multi_class="multinomial", random_state=42
    ),
    "Random Forest": RandomForestClassifier(
        n_estimators=200, random_state=42, n_jobs=-1
    ),
    "Gradient Boosting": GradientBoostingClassifier(
        n_estimators=200, random_state=42
    ),
    "XGBoost": XGBClassifier(
        objective="multi:softmax",
        num_class=4,
        eval_metric="mlogloss",
        random_state=42,
        n_estimators=300,
        use_label_encoder=False  # backward-compatible
    ),
    "SVM (Linear)": LinearSVC(
        random_state=42, max_iter=5000
    )
}


In [ ]:
# Shift severity labels to start from 0
y_train -= 1
y_test  -= 1

print("Unique labels in train:", y_train.unique())
print("Unique labels in test:", y_test.unique())


In [ ]:
# Assuming you have already split your data into X_train, X_test, y_train, y_test

# Combine train and test indices
used_indices = X_train.index.union(X_test.index)

# Remaining/unseen records
X_remaining = df.drop(used_indices).drop('Severity', axis=1)
remaining_df = df.drop(used_indices).copy()

print(f"Number of remaining/unseen records: {len(X_remaining)}")


In [ ]:
# Define models
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
    "Random Forest": RandomForestClassifier(n_estimators=200, random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(n_estimators=200, random_state=42),
    "xgb_model" : XGBClassifier(
    objective='multi:softmax',  
    num_class=4,                
    eval_metric='mlogloss',
    random_state=4),
    "svm_model" : LinearSVC(random_state=42, max_iter=5000)


}

# Dictionary to store results
results = {}

# For ROC curve plotting (binary only)
plt.figure(figsize=(8,6))
binary_target = len(y.unique()) == 2

for name, clf in models.items():
    print(f"\n--------- {name} --------- ")
    
    # Train model
    clf.fit(X_train, y_train)
    
    # Predict
    y_pred = clf.predict(X_test)
    
    # Accuracy & F1
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    print(f"Accuracy: {acc:.4f}")
    print(f"F1-score: {f1:.4f}")
    print(classification_report(y_test, y_pred))
    
    results[name] = (acc, f1)
    
    # ROC-AUC (only if binary)
    if binary_target:
        y_prob = clf.predict_proba(X_test)[:,1]
        roc_auc = roc_auc_score((y_test==y_test.unique()[1]).astype(int), y_prob)
        print(f"ROC-AUC: {roc_auc:.4f}")
        
        # Plot ROC curve
        fpr, tpr, _ = roc_curve((y_test==y_test.unique()[1]).astype(int), y_prob)
        plt.plot(fpr, tpr, label=f"{name} (AUC={roc_auc:.2f})")

# Showing ROC curve (binary only)
if binary_target:
    plt.plot([0,1], [0,1], 'k--')
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("ROC Curve")
    plt.legend()
    plt.show()

# Overview
print("\n -------- Model Comparison (Accuracy / F1)  -------- ")
for k, v in results.items():
    print(f"{k}: Accuracy={v[0]:.4f}, F1={v[1]:.4f}")


In [ ]:
# Model results 
results = {
    "Model": ["Logistic Regression", "Support Vector Machine", 
              "Gradient Boosting", "Random Forest", "XGBoost"],
    "Accuracy": [0.6818, 0.6809, 0.8498, 0.8985, 0.9124],
    "F1 Score": [0.6818, 0.6809, 0.8498, 0.8981, 0.9120]
}

df_results = pd.DataFrame(results)

# Plot leaderboard
plt.figure(figsize=(8,5))
df_results.set_index("Model")[["Accuracy", "F1 Score"]].plot(
    kind="bar", figsize=(10,6), width=0.7
)
plt.title("Model Leaderboard (Accuracy & F1 Score)", fontsize=14)
plt.ylabel("Score")
plt.ylim(0.6, 1.0)
plt.xticks(rotation=30, ha="right")
plt.legend(loc="lower right")
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.show()

### Insight: Ensemble methods, particularly XGBoost, excel in handling structured, high-dimensional accident data, making them ideal for severity prediction.

In [ ]:

# Update your models dictionary to include XGBoost
models = {
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
}

# Define k-fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Random Forest CV
rf_scores = cross_val_score(models['Random Forest'], X_train, y_train, cv=kf, scoring='accuracy')

print("Cross-Validation Results for Random Forest")
print(f"Fold-wise Accuracies : {rf_scores}")
print(f"Mean CV Accuracy     : {rf_scores.mean():.4f}")
print(f"Standard Deviation   : {rf_scores.std():.4f}")

print("=========================================")
# XGBoost CV
xgb_scores = cross_val_score(models['XGBoost'], X_train, y_train, cv=kf, scoring='accuracy')

print("Cross-Validation Results for XGBoost")
print(f"Fold-wise Accuracies : {xgb_scores}")
print(f"Mean CV Accuracy     : {xgb_scores.mean():.4f}")
print(f"Standard Deviation   : {xgb_scores.std():.4f}")



- **Random Forest** and **XGBoost** perform very similarly, both in cross-validation and on unseen test data.  

# Overview of Models and Evalution Matirx
## Models Applied
- **Logistic Regression:** Established as a baseline for interpretability.  
- **Random Forest:** Captured non-linear feature interactions and ranked feature importance.  
- **XGBoost:** Applied as a boosting model to maximize predictive accuracy.  
- **Other Models (if applied):** Compared performance across additional algorithms.

---

## Evaluation Metrics
- **Accuracy:** Overall correctness of predictions.  
- **Precision, Recall, F1-score:** Evaluated class-level performance, critical due to severity imbalance.  
- **Confusion Matrix:** Provided detailed breakdown of misclassifications across severity levels.  
- **ROC-AUC:** Assessed discriminatory power across models.

### Insights
- **XGBoost** outperformed all other models, achieving the best balance of accuracy and recall.  
- **Random Forest** offered strong performance with added interpretability via feature importance.  

In [ ]:
# Feature Importance (Random Forest & XGBoost)
top_models = {
    "Random Forest": RandomForestClassifier(n_estimators=200, random_state=42),
    "XGBoost": XGBClassifier(
        objective='multi:softmax',
        num_class=4,
        eval_metric='mlogloss',
        n_estimators=300,
        random_state=42
    )
}

for name, model in top_models.items():
    print(f"\n=== {name} Feature Importance ===")
    
    # Fit model
    model.fit(X_train, y_train)
    
    # Extract importance
    importances = model.feature_importances_
    features = X_train.columns
    fi_df = pd.DataFrame({
        'Feature': features,
        'Importance': importances
    }).sort_values('Importance', ascending=False).head(15)
    
    # Plot
    plt.figure(figsize=(8,6))
    plt.barh(fi_df['Feature'], fi_df['Importance'], color='blue')
    plt.gca().invert_yaxis()
    plt.title(f"Top 15 Features - {name}")
    plt.xlabel("Importance")
    plt.show()


### Feature Importance Analysis: Random Forest & XGBoost

To identify which features most influence accident severity, computed**feature importance** using our two top-performing models:  

1. **Random Forest** – evaluates feature impact based on tree splits across all trees.  
2. **XGBoost** – gradient boosting method that ranks features by their contribution to model performance.  

For each model, we display the **top most important features** in a horizontal bar chart.  

These plots provide insights into the **key predictors of accident severity** and help guide feature selection and risk mitigation strategies.


In [ ]:
#Precision-Recall Curve
# Binarize test labels for One-vs-Rest
classes = sorted(y_test.unique())
y_test_bin = label_binarize(y_test, classes=classes)
n_classes = y_test_bin.shape[1]
for name, clf in models.items():
    clf.fit(X_train, y_train)
    print(f"{name} model trained successfully!")
# Get predicted probabilities for Random Forest and XGBoost
rf_probs = models['Random Forest'].predict_proba(X_test)
xgb_probs = models['XGBoost'].predict_proba(X_test)

# Precision-Recall Curve
plt.figure(figsize=(10, 6))

# Random Forest PR curves
for i in range(n_classes):
    precision, recall, _ = precision_recall_curve(y_test_bin[:, i], rf_probs[:, i])
    plt.plot(recall, precision, label=f"RF Class {classes[i]}")

# XGBoost PR curves
for i in range(n_classes):
    precision, recall, _ = precision_recall_curve(y_test_bin[:, i], xgb_probs[:, i])
    plt.plot(recall, precision, '--', label=f"XGB Class {classes[i]}")

plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Multiclass Precision-Recall Curve")
plt.legend(loc="lower left", fontsize=8)
plt.grid(alpha=0.3)
plt.show()



In [ ]:
# Create a DataFrame with actual labels and predictions
test_predictions = X_test.copy()  # Start with test features if you want to keep them
test_predictions['Actual'] = y_test
test_predictions['XGBoost_Pred'] = models['XGBoost'].predict(X_test)

# Optional: add predicted probabilities
test_predictions['XGBoost_Prob'] = models['XGBoost'].predict_proba(X_test)[:, 1]  # probability for positive class

# Save to CSV
test_predictions.to_csv('test_predictions.csv', index=False)

print("Test predictions saved successfully to 'test_predictions.csv'")

In [ ]:
# Get predictions
y_pred = models['XGBoost'].predict(X_test)

# Accuracy
accuracy = accuracy_score(y_test, y_pred)

# Balanced Accuracy
balanced_acc = balanced_accuracy_score(y_test, y_pred)

# Precision, Recall, F1 (macro-averaged for multiclass)
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Note: Specificity is not directly defined for multiclass; you can compute it per class if needed
# ROC-AUC for multiclass requires predicted probabilities and One-vs-Rest
from sklearn.preprocessing import label_binarize
y_test_bin = label_binarize(y_test, classes=sorted(y_test.unique()))
y_proba = models['XGBoost'].predict_proba(X_test)
roc_auc = roc_auc_score(y_test_bin, y_proba, average='macro', multi_class='ovr')


print("Test Data Mertics: ")
print('=='*30)
# Store metrics
test_metrics = {
    "Accuracy": accuracy,
    "Balanced Accuracy": balanced_acc,
    "Precision (macro)": precision,
    "Recall (macro)": recall,
    "F1 Score (macro)": f1,
    "ROC-AUC (macro, OvR)": roc_auc,

    "Confusion Matrix": cm
}

test_metrics


In [ ]:
# Binarize labels for multiclass (One-vs-Rest)
classes = sorted(y_test.unique())
y_test_bin = label_binarize(y_test, classes=classes)
y_proba = models['XGBoost'].predict_proba(X_test)
n_classes = y_test_bin.shape[1]

# ROC Curve per class
plt.figure(figsize=(10, 5))
for i in range(n_classes):
    fpr, tpr, _ = roc_curve(y_test_bin[:, i], y_proba[:, i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f"Class {classes[i]} (AUC = {roc_auc:.3f})")
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve - Test Data")
plt.legend(loc="lower right", fontsize=8)
plt.grid(alpha=0.3)
plt.show()

# Precision-Recall Curve per class
plt.figure(figsize=(10, 5))
for i in range(n_classes):
    precision, recall, _ = precision_recall_curve(y_test_bin[:, i], y_proba[:, i])
    ap = average_precision_score(y_test_bin[:, i], y_proba[:, i])
    plt.plot(recall, precision, label=f"Class {classes[i]} (AP = {ap:.3f})")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision-Recall Curve - Test Data")
plt.legend(loc="lower right", fontsize=8)
plt.grid(alpha=0.3)
plt.show()

# Test Set Model Evaluation (ROC & Precision-Recall)

Evaluated the final XGBoost model on the **test dataset** using both **ROC curves** and **Precision-Recall (PR) curves** to assess performance for each class.

---

## ROC Curve

- The **ROC curve** shows the model's ability to distinguish each class from the rest (One-vs-Rest).  
- **AUC (Area Under Curve)** is reported for each class, indicating discrimination power.  
- Higher AUC values (~1.0) indicate better ability to separate that class from others.  
- Visual inspection helps identify which classes are more difficult for the model to distinguish.

---

## Precision-Recall Curve

- The **PR curve** highlights the trade-off between **precision** and **recall** for each class.  
- **Average Precision (AP)** is reported per class, summarizing the area under each PR curve.  
- Particularly useful for **imbalanced classes**, showing how well the model identifies positives.  
- Classes with lower AP may require further attention, e.g., class weighting or resampling.

---

### Key Takeaways

1. **ROC-AUC values** indicate strong discrimination for most classes.  
2. **PR curves** reveal that the model performs very well for majority classes but may struggle with minority classes.  
3. These visualizations complement the numeric metrics (accuracy, precision, recall, F1) to give a complete understanding of model performance on the test data.


The predictions have been saved to `test_predictions.csv` for further analysis or deployment.


### Conclusion & Next Steps

This analysis explored the US Accidents dataset with a focus on understanding and predicting accident severity. After rigorous **data cleaning** and **feature engineering**, including temporal (`Hour`, `DayOfWeek`, `Rush_Hour`), environmental (`Weather_Condition`, `Visibility_Precip`), and location-based features, we trained multiple machine learning models. **Random Forest** and **XGBoost** consistently outperformed others, achieving the highest accuracy and F1-scores. Feature importance analysis highlighted that **peak hours, adverse weather conditions, and low visibility** are the strongest predictors of accident severity, confirming domain insights.  

**Next Steps & Recommendations:**  
-  **Hyperparameter Optimization:** Fine-tune Random Forest and XGBoost to further boost predictive performance.  
-  **Class Imbalance Handling:** Apply SMOTE or class-weight adjustments for rare severity classes.  
-  **Geospatial Risk Mapping:** Cluster accident locations to identify hotspots and inform preventive measures.  
-  **Deployment & Real-Time Prediction:** Integrate the trained pipeline into a dashboard for live accident risk assessment.  
-  **Enhanced Feature Engineering:** Incorporate traffic density, road type, and historical accident trends to refine predictions.  

This workflow demonstrates a **complete ML pipeline** from raw data to interpretable insights, ready for **real-world application and further experimentation**.
